In [1]:
import pandas as pd

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train.columns

Index(['Severity', 'Safety_Score', 'Days_Since_Inspection',
       'Total_Safety_Complaints', 'Control_Metric', 'Turbulence_In_gforces',
       'Cabin_Temperature', 'Accident_Type_Code', 'Max_Elevation',
       'Violations', 'Adverse_Weather_Metric', 'Accident_ID'],
      dtype='object')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
Severity                   10000 non-null object
Safety_Score               10000 non-null float64
Days_Since_Inspection      10000 non-null int64
Total_Safety_Complaints    10000 non-null int64
Control_Metric             10000 non-null float64
Turbulence_In_gforces      10000 non-null float64
Cabin_Temperature          10000 non-null float64
Accident_Type_Code         10000 non-null int64
Max_Elevation              10000 non-null float64
Violations                 10000 non-null int64
Adverse_Weather_Metric     10000 non-null float64
Accident_ID                10000 non-null int64
dtypes: float64(6), int64(5), object(1)
memory usage: 937.6+ KB


In [5]:
train.describe()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000
mean,41.876406,12.931100,6.564300,65.145324,0.381495,79.969331,3.814900,32001.803282,2.01220,0.255635,6266.554200
std,16.138072,3.539803,6.971982,11.882934,0.121301,2.759739,1.902577,9431.995196,1.03998,0.381128,3610.170288
min,0.000000,1.000000,0.000000,0.000000,0.134000,74.740000,1.000000,831.695553,0.00000,0.000316,2.000000
25%,30.593607,11.000000,2.000000,56.927985,0.293665,77.960000,2.000000,25757.636908,1.00000,0.012063,3139.750000
50%,41.278539,13.000000,4.000000,65.587967,0.365879,79.540000,4.000000,32060.336419,2.00000,0.074467,6280.500000
75%,52.511416,15.000000,9.000000,73.336372,0.451346,81.560000,5.000000,38380.641513,3.00000,0.354059,9391.500000
max,100.000000,23.000000,54.000000,100.000000,0.882648,97.510000,7.000000,64297.651218,5.00000,2.365378,12500.000000


In [6]:
train['Accident_Type_Code'].replace(to_replace=7,value=0,inplace=True)
test['Accident_Type_Code'].replace(to_replace=7,value=0,inplace=True)

In [7]:
from sklearn.preprocessing import PowerTransformer
log=PowerTransformer()
log.fit(train[['Max_Elevation']])
train['Max_Elevation']=log.transform(train[['Max_Elevation']])
test['Max_Elevation']=log.transform(test[['Max_Elevation']])

In [8]:
from sklearn.preprocessing import MinMaxScaler
mms=MinMaxScaler()

In [9]:
train.columns

Index(['Severity', 'Safety_Score', 'Days_Since_Inspection',
       'Total_Safety_Complaints', 'Control_Metric', 'Turbulence_In_gforces',
       'Cabin_Temperature', 'Accident_Type_Code', 'Max_Elevation',
       'Violations', 'Adverse_Weather_Metric', 'Accident_ID'],
      dtype='object')

In [10]:
features=train.drop(['Accident_ID','Severity','Max_Elevation','Adverse_Weather_Metric'],axis=1)
y=train['Severity']

In [11]:
mms.fit(features)
x=mms.transform(features)

In [12]:
test_x=test.drop(['Accident_ID','Max_Elevation','Adverse_Weather_Metric'],axis=1)
test_x=mms.transform(test_x)

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=1111)

In [14]:
from sklearn.metrics import make_scorer,f1_score
ms=make_scorer(f1_score,average='weighted')

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
xgb = GradientBoostingClassifier(max_depth=4,n_estimators=500 ,random_state=1111)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
y_hat = xgb.predict(X_train)

In [16]:
train_score=f1_score(y_train,y_hat,average='weighted')
test_score=f1_score(y_test,y_pred,average='weighted')
print("Training set score:",train_score)
print("Test set score:",test_score)

Training set score: 1.0
Test set score: 0.9664937105893476


In [17]:
result=xgb.predict(test_x)

In [18]:
sub=pd.DataFrame(data={'Accident_ID':test['Accident_ID'] })

In [19]:
sub['Severity']=result

In [20]:
sub.head()

,Accident_ID,Severity
0,1,Highly_Fatal_And_Damaging
1,10,Significant_Damage_And_Fatalities
2,14,Significant_Damage_And_Serious_Injuries
3,17,Highly_Fatal_And_Damaging
4,21,Significant_Damage_And_Fatalities


In [21]:
sub.shape

(2500, 2)